## Import Library & Packages

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, FunctionTransformer
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering, KMeans
from yellowbrick.cluster import KElbowVisualizer
from datetime import datetime
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LassoCV

dataset URL: https://www.kaggle.com/datasets/imakash3011/customer-personality-analysis

# Pipeline

In [3]:
# Data Import Function
def import_data(path):
    return pd.read_csv(path, sep='\t')

In [4]:
# Missing Values Imputation Function
def impute_missing(data):
    columns_for_imputation = ['Year_Birth', 'Kidhome', 'Teenhome', 'Income']
    data_for_imputation = data[columns_for_imputation].copy()

    data_categorical = data[['Education', 'Marital_Status']]

    encoder = OneHotEncoder(drop='first')
    data_encoded = encoder.fit_transform(data_categorical) 

    data_encoded = pd.DataFrame(data_encoded.toarray(), 
                                columns=encoder.get_feature_names_out(['Education', 'Marital_Status']))

    data_for_imputation = data_for_imputation.reset_index(drop=True)
    data_encoded = data_encoded.reset_index(drop=True)

    data_combined = pd.concat([data_for_imputation, data_encoded], axis=1)

    imputer = IterativeImputer()
    imputed_data = pd.DataFrame(imputer.fit_transform(data_combined), columns=data_combined.columns)

    data.reset_index(drop=True, inplace=True)
    imputed_data.reset_index(drop=True, inplace=True)
    data.loc[data['Income'].isnull(), 'Income'] = imputed_data.loc[data['Income'].isnull(), 'Income']
    return data

In [5]:
# Outliers Removal Function
def remove_outliers(data):
    data = data[~data['Year_Birth'].isin([1893, 1899, 1900])]
    data = data[(data["Income"] < 160000)] 
    return data

In [6]:
# Feature Engineering Function
def feature_engineering(data):
    data_newfeature = data.copy()

    # Membership enrollment time !! SHOULD WE USE A DATE INSTEAD?
    data_newfeature['Enroll_Year'] = data_newfeature['Dt_Customer'].str[-4:].astype(int)
    data_newfeature['Enroll_Month'] = data_newfeature['Dt_Customer'].str[3:5].astype(int)
    data_newfeature['Enroll_Day'] = data_newfeature['Dt_Customer'].str[0:2].astype(int)
    data_newfeature.drop(columns=['Dt_Customer'], inplace=True)
    data_newfeature['Member_Year'] = 2016 - data_newfeature['Enroll_Year']

    # Since max enroll_year is 2014, we assume the company is evaluating their customer in 2016
    data_newfeature['Age'] = 2016 - data_newfeature['Year_Birth']     

    # Add total amount
    data_newfeature['Total_amount'] = data_newfeature[['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 
                                            'MntSweetProducts', 'MntGoldProds']].sum(axis=1)

    # Add total purchase
    data_newfeature['Total_purchase'] = data_newfeature[['NumWebPurchases', 'NumCatalogPurchases', 
                                            'NumStorePurchases']].sum(axis=1)

    # Calculate features regarding the user
    data_newfeature['Total_Children'] = data_newfeature['Kidhome'] + data_newfeature['Teenhome']
    data_newfeature["Is_Parent"] = np.where(data_newfeature['Total_Children']> 0, 1, 0)

    data_newfeature["Family_Size"] = data_newfeature["Marital_Status"].replace({"Married": 2, "Together": 2, \
                                "Absurd": 1, "Widow": 1, "YOLO": 1, "Divorced": 1, "Single": 1, "Alone": 1}).astype(int) \
                                + data_newfeature["Total_Children"]


    data_newfeature['Cmp_Attitude'] = np.where((data_newfeature['AcceptedCmp1'] + data_newfeature['AcceptedCmp2'] +
                                                data_newfeature['AcceptedCmp3'] + data_newfeature['AcceptedCmp4'] +
                                                data_newfeature['AcceptedCmp5']) > 0, 1, 0)

    # Calculate the percentiles for 'Recency', 'Frequency', and 'Monetary'
    recency_50th = data_newfeature['Recency'].quantile(0.5)
    frequency_50th = data_newfeature['Total_purchase'].quantile(0.5)
    monetary_50th = data_newfeature['Total_amount'].quantile(0.5)

    # Assign binary scores for each R,F,M category
    data_newfeature['R'] = (data_newfeature['Recency'] <= recency_50th).astype(int)
    data_newfeature['F'] = (data_newfeature['Total_purchase'] > frequency_50th).astype(int)
    data_newfeature['M'] = (data_newfeature['Total_amount'] > monetary_50th).astype(int)

    # Combine the R,F,M scores to a single group identifier
    data_newfeature['RFM_Score'] = data_newfeature['R'].astype(str) + data_newfeature['F'].astype(str) +\
        data_newfeature['M'].astype(str)

    # Define group names based on RFM_Score
    group_names = {
        '111': 'Loyalist',
        '011': 'Potential Loyalist',
        '101': 'New Customers',
        '001': 'Promising',
        '110': 'At Risk',
        '010': 'Need Attention',
        '100': 'About To Sleep',
        '000': 'Hibernating'
    }
        
    # Map the group names to the dataframe
    data_newfeature['RFM_Group'] = data_newfeature['RFM_Score'].map(group_names)
        
        # One-hot encoding the 'RFM_Group'
    data_newfeature = pd.get_dummies(data_newfeature, columns=['RFM_Group'])

    # Initialize a new column with empty strings
    data_newfeature['Age_Demographic'] = ""
        
        # Iterate over each row to assign the age demographic based on age
    for index, row in data_newfeature.iterrows():
        if row["Year_Birth"] <= 1964:
            data_newfeature.at[index, "Age_Demographic"] = "Baby Boomer"
        elif row["Year_Birth"] <= 1980:
            data_newfeature.at[index, "Age_Demographic"] = "Gen X"
        elif row["Year_Birth"] <= 1996:
            data_newfeature.at[index, "Age_Demographic"] = "Gen Y"
        else:
            data_newfeature.at[index, "Age_Demographic"] = "Gen Z"
    data_newfeature = pd.get_dummies(data_newfeature, columns=['Age_Demographic'])

    # Iterate over each row to assign the income category based on income
    for index, row in data_newfeature.iterrows():
        if row["Income"] <= 2333:
            data_newfeature.at[index, "Income_Category"] = "Low"
        elif row["Income"] <= 12427:
            data_newfeature.at[index, "Income_Category"] = "Medium"
        else:
            data_newfeature.at[index, "Income_Category"] = "High"
        
    data_newfeature = pd.get_dummies(data_newfeature, columns=['Income_Category'])

    # Drop the columns that are not needed
    data_newfeature.drop(columns=['Z_CostContact', 'Z_Revenue'], inplace=True)

    return data_newfeature

In [7]:
# One-Hot Encoding Function
def one_hot_encoding(data):
    encoder = OneHotEncoder(drop='first', sparse_output=False)

    encoded_data = encoder.fit_transform(data[['Education', 'Marital_Status']])

    encoded_feature_names = encoder.get_feature_names_out(['Education', 'Marital_Status'])

    encoded_df = pd.DataFrame(encoded_data, columns=encoded_feature_names)

    data.reset_index(drop=True, inplace=True)
    encoded_df.reset_index(drop=True, inplace=True)

    data = pd.concat([data.drop(columns=['Education', 'Marital_Status', 'RFM_Score']), encoded_df], axis=1)

    return data


In [8]:
# Pipeline Definition
preprocessing_pipeline = Pipeline([
    ('import_data', FunctionTransformer(import_data)),
    ('impute_missing', FunctionTransformer(impute_missing)),
    ('remove_outliers', FunctionTransformer(remove_outliers)),
    ('feature_engineering', FunctionTransformer(feature_engineering)),
    ('one_hot_encoding', FunctionTransformer(one_hot_encoding))
])

In [9]:
path = '/Users/qianlou/Documents/GitHub/Customer-Personality-Analysis-2.0/Data/Raw Data/marketing_campaign.csv'
processed_data = preprocessing_pipeline.transform(path)

/var/folders/z4/djkh1_vd3bb6xf3hy80b_l9m0000gn/T/ipykernel_73069/274487818.py:27: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_newfeature["Family_Size"] = data_newfeature["Marital_Status"].replace({"Married": 2, "Together": 2, \


# Feature Selection

## LassoCV

In [10]:
def feature_selection_lasso(df, target_column):
    # Drop columns that are not needed
    df = df.drop(columns=['ID', 'Year_Birth', 'Kidhome', 'Teenhome', 'AcceptedCmp1', 'AcceptedCmp2',
                          'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Enroll_Year', 'Enroll_Month',
                          'Enroll_Day', 'Member_Year', 'Total_Children', 'Is_Parent', 'Family_Size',
                          'RFM_Group_About To Sleep', 'RFM_Group_At Risk', 'RFM_Group_Hibernating',
                          'RFM_Group_Loyalist', 'RFM_Group_Need Attention', 'RFM_Group_New Customers',
                          'RFM_Group_Potential Loyalist', 'RFM_Group_Promising', 'Age_Demographic_Baby Boomer',
                          'Age_Demographic_Gen X', 'Age_Demographic_Gen Y', 'Income'])

    if target_column == 'Total_purchase':
        df = df.drop(columns=['NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumDealsPurchases',
                              'Total_amount'])
    elif target_column == 'Total_amount':
        df = df.drop(columns=['MntWines', 'MntFruits', 'MntMeatProducts', 'MntFishProducts', 
                              'MntSweetProducts', 'MntGoldProds', 'Total_purchase'])

    # Separate the features and the target
    X = df.drop(columns=[target_column])
    y = df[target_column]
            
    # Normalize the features
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
            
    # Perform feature selection using LassoCV
    lasso = LassoCV(cv=5).fit(X_scaled, y)
            
    # Get the coefficients
    coef = lasso.coef_
            
    # Select features that have non-zero coefficients
    selected_features = X.columns[coef != 0]
            
    # Create a new dataframe with selected features and normalize them
    df_selected = df[selected_features]
    df_selected = pd.DataFrame(scaler.fit_transform(df_selected), columns=df_selected.columns)
            
    # Add the target column back
    df_selected[target_column] = y
            
    return df_selected

### Target column: 'Response' (whether accepted the offer in the recent campaign)

In [11]:
target_column = 'Response'
data_selected = feature_selection_lasso(processed_data, target_column)
data_selected.sort_values(by=target_column, ascending=False)
data_selected.reset_index(drop=True, inplace=True)
data_selected.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 30 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Recency                  2234 non-null   float64
 1   MntFruits                2234 non-null   float64
 2   MntMeatProducts          2234 non-null   float64
 3   MntFishProducts          2234 non-null   float64
 4   MntSweetProducts         2234 non-null   float64
 5   MntGoldProds             2234 non-null   float64
 6   NumDealsPurchases        2234 non-null   float64
 7   NumWebPurchases          2234 non-null   float64
 8   NumCatalogPurchases      2234 non-null   float64
 9   NumStorePurchases        2234 non-null   float64
 10  NumWebVisitsMonth        2234 non-null   float64
 11  Complain                 2234 non-null   float64
 12  Age                      2234 non-null   float64
 13  Total_amount             2234 non-null   float64
 14  Cmp_Attitude            

In [12]:
data_selected.describe()

,Recency,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,...,Education_Basic,Education_Master,Education_PhD,Marital_Status_Alone,Marital_Status_Married,Marital_Status_Single,Marital_Status_Together,Marital_Status_Widow,Marital_Status_YOLO,Response
count,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,...,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000,2.234000e+03,2234.000000
mean,3.816702e-17,-4.691363e-17,1.908351e-17,2.942041e-17,-2.226410e-17,4.293790e-17,6.997287e-17,8.587580e-17,-1.908351e-17,-1.196695e-16,...,2.544468e-17,-3.498644e-17,4.770878e-17,-6.361170e-18,1.049593e-16,5.725053e-17,6.281656e-17,0.000000,1.272234e-17,0.149508
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,...,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224,1.000224e+00,0.356668
min,-1.697005e+00,-6.617849e-01,-7.441592e-01,-6.872581e-01,-6.562155e-01,-8.455772e-01,-1.212761e+00,-1.472800e+00,-9.229950e-01,-1.785309e+00,...,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-7.933899e-01,-5.224313e-01,-5.901014e-01,-0.188938,-2.993422e-02,0.000000
25%,-8.681360e-01,-6.366142e-01,-6.726053e-01,-6.323672e-01,-6.320035e-01,-6.726956e-01,-6.903338e-01,-7.528363e-01,-9.229950e-01,-8.618592e-01,...,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-7.933899e-01,-5.224313e-01,-5.901014e-01,-0.188938,-2.993422e-02,0.000000
50%,1.253754e-02,-4.604191e-01,-4.445273e-01,-4.676945e-01,-4.625199e-01,-3.845596e-01,-1.679064e-01,-3.287213e-02,-2.272059e-01,-2.462258e-01,...,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-7.933899e-01,-5.224313e-01,-5.901014e-01,-0.188938,-2.993422e-02,0.000000
75%,8.586749e-01,1.688489e-01,2.922541e-01,2.275899e-01,1.427790e-01,2.301304e-01,3.545210e-01,6.870920e-01,4.685831e-01,6.772243e-01,...,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,1.260414e+00,-5.224313e-01,1.694624e+00,-0.188938,-2.993422e-02,0.000000
max,1.722080e+00,4.347188e+00,6.970244e+00,4.051655e+00,5.711528e+00,6.108104e+00,6.623650e+00,8.246715e+00,8.818051e+00,2.216308e+00,...,6.353768e+00,2.244513e+00,1.904011e+00,2.727025e+01,1.260414e+00,1.914127e+00,1.694624e+00,5.292730,3.340659e+01,1.000000


### Target column: 'Total_purchase' (total number of purchase)

In [13]:
target_column = 'Total_purchase'
data_selected2 = feature_selection_lasso(processed_data, target_column)
data_selected2.sort_values(by=target_column, ascending=False)
data_selected2.reset_index(drop=True, inplace=True)
data_selected2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   MntWines               2234 non-null   float64
 1   MntFruits              2234 non-null   float64
 2   MntMeatProducts        2234 non-null   float64
 3   MntFishProducts        2234 non-null   float64
 4   MntSweetProducts       2234 non-null   float64
 5   MntGoldProds           2234 non-null   float64
 6   Response               2234 non-null   float64
 7   Age                    2234 non-null   float64
 8   Cmp_Attitude           2234 non-null   float64
 9   R                      2234 non-null   float64
 10  F                      2234 non-null   float64
 11  M                      2234 non-null   float64
 12  Income_Category_High   2234 non-null   float64
 13  Income_Category_Low    2234 non-null   float64
 14  Education_Basic        2234 non-null   float64
 15  Mari

In [14]:
data_selected2.describe()

,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,Response,Age,Cmp_Attitude,R,F,M,Income_Category_High,Income_Category_Low,Education_Basic,Marital_Status_Single,Marital_Status_YOLO,Total_purchase
count,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000,2.234000e+03,2234.000000,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000
mean,9.223697e-17,-4.691363e-17,1.908351e-17,2.942041e-17,-2.226410e-17,4.293790e-17,4.293790e-17,2.711449e-16,-4.850392e-17,0.000000,-5.884083e-17,0.000000,1.129108e-16,3.180585e-18,2.544468e-17,5.725053e-17,1.272234e-17,12.544315
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224,1.000224e+00,1.000224,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,7.197244
min,-9.041670e-01,-6.617849e-01,-7.441592e-01,-6.872581e-01,-6.562155e-01,-8.455772e-01,-4.192726e-01,-2.316854e+00,-5.106098e-01,-1.000000,-9.884286e-01,-1.000000,-7.813805e+00,-2.116195e-02,-1.573869e-01,-5.224313e-01,-2.993422e-02,0.000000
25%,-8.328644e-01,-6.366142e-01,-6.726053e-01,-6.323672e-01,-6.320035e-01,-6.726956e-01,-4.192726e-01,-6.931512e-01,-5.106098e-01,-1.000000,-9.884286e-01,-1.000000,1.279786e-01,-2.116195e-02,-1.573869e-01,-5.224313e-01,-2.993422e-02,6.000000
50%,-3.857380e-01,-4.604191e-01,-4.445273e-01,-4.676945e-01,-4.625199e-01,-3.845596e-01,-4.192726e-01,-9.494488e-02,-5.106098e-01,0.000000,-9.884286e-01,0.000000,1.279786e-01,-2.116195e-02,-1.573869e-01,-5.224313e-01,-2.993422e-02,12.000000
75%,5.954147e-01,1.688489e-01,2.922541e-01,2.275899e-01,1.427790e-01,2.301304e-01,-4.192726e-01,8.450936e-01,-5.106098e-01,1.000000,1.011707e+00,1.000000,1.279786e-01,-2.116195e-02,-1.573869e-01,-5.224313e-01,-2.993422e-02,18.000000
max,3.531445e+00,4.347188e+00,6.970244e+00,4.051655e+00,5.711528e+00,6.108104e+00,2.385083e+00,2.468796e+00,1.958443e+00,1.000000,1.011707e+00,1.000000,1.279786e-01,4.725463e+01,6.353768e+00,1.914127e+00,3.340659e+01,32.000000


### Target column: 'Total_amount' (total amount spent)

In [15]:
target_column = 'Total_amount'
data_selected3 = feature_selection_lasso(processed_data, target_column)
data_selected3.sort_values(by=target_column, ascending=False)
data_selected3.reset_index(drop=True, inplace=True)
data_selected3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Recency                  2234 non-null   float64
 1   NumDealsPurchases        2234 non-null   float64
 2   NumWebPurchases          2234 non-null   float64
 3   NumCatalogPurchases      2234 non-null   float64
 4   NumStorePurchases        2234 non-null   float64
 5   NumWebVisitsMonth        2234 non-null   float64
 6   Complain                 2234 non-null   float64
 7   Response                 2234 non-null   float64
 8   Age                      2234 non-null   float64
 9   Cmp_Attitude             2234 non-null   float64
 10  M                        2234 non-null   float64
 11  Income_Category_High     2234 non-null   float64
 12  Income_Category_Low      2234 non-null   float64
 13  Education_Basic          2234 non-null   float64
 14  Education_Graduation    

In [16]:
data_selected3.describe()

,Recency,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,Complain,Response,Age,Cmp_Attitude,...,Income_Category_High,Income_Category_Low,Education_Basic,Education_Graduation,Education_Master,Marital_Status_Alone,Marital_Status_Married,Marital_Status_Together,Marital_Status_YOLO,Total_amount
count,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,...,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000
mean,3.816702e-17,6.997287e-17,8.587580e-17,-1.908351e-17,-1.196695e-16,7.633405e-17,2.226410e-17,4.293790e-17,2.711449e-16,-4.850392e-17,...,1.129108e-16,3.180585e-18,2.544468e-17,-9.541756e-18,-3.498644e-17,-6.361170e-18,1.049593e-16,6.281656e-17,1.272234e-17,605.712623
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,...,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,601.582708
min,-1.697005e+00,-1.212761e+00,-1.472800e+00,-9.229950e-01,-1.785309e+00,-2.196857e+00,-9.504432e-02,-4.192726e-01,-2.316854e+00,-5.106098e-01,...,-7.813805e+00,-2.116195e-02,-1.573869e-01,-1.008090e+00,-4.455310e-01,-3.666999e-02,-7.933899e-01,-5.901014e-01,-2.993422e-02,5.000000
25%,-8.681360e-01,-6.903338e-01,-7.528363e-01,-9.229950e-01,-8.618592e-01,-9.587694e-01,-9.504432e-02,-4.192726e-01,-6.931512e-01,-5.106098e-01,...,1.279786e-01,-2.116195e-02,-1.573869e-01,-1.008090e+00,-4.455310e-01,-3.666999e-02,-7.933899e-01,-5.901014e-01,-2.993422e-02,69.000000
50%,1.253754e-02,-1.679064e-01,-3.287213e-02,-2.272059e-01,-2.462258e-01,2.793178e-01,-9.504432e-02,-4.192726e-01,-9.494488e-02,-5.106098e-01,...,1.279786e-01,-2.116195e-02,-1.573869e-01,9.919749e-01,-4.455310e-01,-3.666999e-02,-7.933899e-01,-5.901014e-01,-2.993422e-02,396.500000
75%,8.586749e-01,3.545210e-01,6.870920e-01,4.685831e-01,6.772243e-01,6.920135e-01,-9.504432e-02,-4.192726e-01,8.450936e-01,-5.106098e-01,...,1.279786e-01,-2.116195e-02,-1.573869e-01,9.919749e-01,-4.455310e-01,-3.666999e-02,1.260414e+00,1.694624e+00,-2.993422e-02,1044.750000
max,1.722080e+00,6.623650e+00,8.246715e+00,8.818051e+00,2.216308e+00,6.057058e+00,1.052141e+01,2.385083e+00,2.468796e+00,1.958443e+00,...,1.279786e-01,4.725463e+01,6.353768e+00,9.919749e-01,2.244513e+00,2.727025e+01,1.260414e+00,1.694624e+00,3.340659e+01,2525.000000


### Target column: 'Cmp_Attitude' (whether accepted any deal from campaigns)

In [17]:
target_column = 'Cmp_Attitude'
data_selected4 = feature_selection_lasso(processed_data, target_column)
data_selected4.sort_values(by=target_column, ascending=False)
data_selected4.reset_index(drop=True, inplace=True)
data_selected4.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 19 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Recency                  2234 non-null   float64
 1   MntWines                 2234 non-null   float64
 2   MntFruits                2234 non-null   float64
 3   MntGoldProds             2234 non-null   float64
 4   NumDealsPurchases        2234 non-null   float64
 5   NumCatalogPurchases      2234 non-null   float64
 6   NumStorePurchases        2234 non-null   float64
 7   Complain                 2234 non-null   float64
 8   Response                 2234 non-null   float64
 9   Age                      2234 non-null   float64
 10  Education_Basic          2234 non-null   float64
 11  Education_Master         2234 non-null   float64
 12  Education_PhD            2234 non-null   float64
 13  Marital_Status_Alone     2234 non-null   float64
 14  Marital_Status_Divorced 

In [18]:
data_selected4.describe()

,Recency,MntWines,MntFruits,MntGoldProds,NumDealsPurchases,NumCatalogPurchases,NumStorePurchases,Complain,Response,Age,Education_Basic,Education_Master,Education_PhD,Marital_Status_Alone,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Marital_Status_YOLO,Cmp_Attitude
count,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000
mean,3.816702e-17,9.223697e-17,-4.691363e-17,4.293790e-17,6.997287e-17,-1.908351e-17,-1.196695e-16,2.226410e-17,4.293790e-17,2.711449e-16,2.544468e-17,-3.498644e-17,4.770878e-17,-6.361170e-18,-3.021556e-17,1.049593e-16,5.725053e-17,1.272234e-17,0.206804
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,0.405104
min,-1.697005e+00,-9.041670e-01,-6.617849e-01,-8.455772e-01,-1.212761e+00,-9.229950e-01,-1.785309e+00,-9.504432e-02,-4.192726e-01,-2.316854e+00,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-3.395983e-01,-7.933899e-01,-5.224313e-01,-2.993422e-02,0.000000
25%,-8.681360e-01,-8.328644e-01,-6.366142e-01,-6.726956e-01,-6.903338e-01,-9.229950e-01,-8.618592e-01,-9.504432e-02,-4.192726e-01,-6.931512e-01,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-3.395983e-01,-7.933899e-01,-5.224313e-01,-2.993422e-02,0.000000
50%,1.253754e-02,-3.857380e-01,-4.604191e-01,-3.845596e-01,-1.679064e-01,-2.272059e-01,-2.462258e-01,-9.504432e-02,-4.192726e-01,-9.494488e-02,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-3.395983e-01,-7.933899e-01,-5.224313e-01,-2.993422e-02,0.000000
75%,8.586749e-01,5.954147e-01,1.688489e-01,2.301304e-01,3.545210e-01,4.685831e-01,6.772243e-01,-9.504432e-02,-4.192726e-01,8.450936e-01,-1.573869e-01,-4.455310e-01,-5.252070e-01,-3.666999e-02,-3.395983e-01,1.260414e+00,-5.224313e-01,-2.993422e-02,0.000000
max,1.722080e+00,3.531445e+00,4.347188e+00,6.108104e+00,6.623650e+00,8.818051e+00,2.216308e+00,1.052141e+01,2.385083e+00,2.468796e+00,6.353768e+00,2.244513e+00,1.904011e+00,2.727025e+01,2.944655e+00,1.260414e+00,1.914127e+00,3.340659e+01,1.000000


### Target column: 'Recency' (Number of days since customer's last purchase)

In [19]:
target_column = 'Recency'
data_selected5 = feature_selection_lasso(processed_data, target_column)
data_selected5.sort_values(by=target_column, ascending=False)
data_selected5.reset_index(drop=True, inplace=True)
data_selected5.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2234 entries, 0 to 2233
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   MntMeatProducts       2234 non-null   float64
 1   MntGoldProds          2234 non-null   float64
 2   NumWebVisitsMonth     2234 non-null   float64
 3   Complain              2234 non-null   float64
 4   Response              2234 non-null   float64
 5   Age                   2234 non-null   float64
 6   Cmp_Attitude          2234 non-null   float64
 7   R                     2234 non-null   float64
 8   F                     2234 non-null   float64
 9   Income_Category_High  2234 non-null   float64
 10  Education_Graduation  2234 non-null   float64
 11  Education_Master      2234 non-null   float64
 12  Marital_Status_Alone  2234 non-null   float64
 13  Marital_Status_Widow  2234 non-null   float64
 14  Marital_Status_YOLO   2234 non-null   float64
 15  Recency              

In [20]:
data_selected5.describe()

,MntMeatProducts,MntGoldProds,NumWebVisitsMonth,Complain,Response,Age,Cmp_Attitude,R,F,Income_Category_High,Education_Graduation,Education_Master,Marital_Status_Alone,Marital_Status_Widow,Marital_Status_YOLO,Recency
count,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2.234000e+03,2234.000000,2.234000e+03,2234.000000
mean,1.908351e-17,4.293790e-17,7.633405e-17,2.226410e-17,4.293790e-17,2.711449e-16,-4.850392e-17,0.000000,-5.884083e-17,1.129108e-16,-9.541756e-18,-3.498644e-17,-6.361170e-18,0.000000,1.272234e-17,49.136974
std,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224e+00,1.000224,1.000224e+00,28.961593
min,-7.441592e-01,-8.455772e-01,-2.196857e+00,-9.504432e-02,-4.192726e-01,-2.316854e+00,-5.106098e-01,-1.000000,-9.884286e-01,-7.813805e+00,-1.008090e+00,-4.455310e-01,-3.666999e-02,-0.188938,-2.993422e-02,0.000000
25%,-6.726053e-01,-6.726956e-01,-9.587694e-01,-9.504432e-02,-4.192726e-01,-6.931512e-01,-5.106098e-01,-1.000000,-9.884286e-01,1.279786e-01,-1.008090e+00,-4.455310e-01,-3.666999e-02,-0.188938,-2.993422e-02,24.000000
50%,-4.445273e-01,-3.845596e-01,2.793178e-01,-9.504432e-02,-4.192726e-01,-9.494488e-02,-5.106098e-01,0.000000,-9.884286e-01,1.279786e-01,9.919749e-01,-4.455310e-01,-3.666999e-02,-0.188938,-2.993422e-02,49.500000
75%,2.922541e-01,2.301304e-01,6.920135e-01,-9.504432e-02,-4.192726e-01,8.450936e-01,-5.106098e-01,1.000000,1.011707e+00,1.279786e-01,9.919749e-01,-4.455310e-01,-3.666999e-02,-0.188938,-2.993422e-02,74.000000
max,6.970244e+00,6.108104e+00,6.057058e+00,1.052141e+01,2.385083e+00,2.468796e+00,1.958443e+00,1.000000,1.011707e+00,1.279786e-01,9.919749e-01,2.244513e+00,2.727025e+01,5.292730,3.340659e+01,99.000000


# Export Preprocessed data

In [21]:
processed_data.to_csv('Final Preprocessed Data.csv', index=False)